In [1]:
import numpy as np
import copy

In [2]:
def Sigmoid(x):
    res = 1 / (1 + np.exp(-x))
    return res

def d_Sigmoid(x):
    y = x * (1 - x)
    return y

def ReLU(x):
    x = np.maximum(0, x)
    return x
    
def d_ReLU(x): 
    x[x<=0] = 0
    x[x>0] = 1
    return x


def Linear(x):
    return x

def d_Linear(x):
    y = np.ones(shape=(x.shape), dtype = x.dtype)
    return y

activations_dict = {
'Sigmoid': [Sigmoid, d_Sigmoid],
'ReLU': [ReLU, d_ReLU], 
'Linear': [Linear, d_Linear]
}


In [3]:
class layer:
    def __init__(s, lr = 0.1, prev_size = 2, my_size=2, activation_type = "Sigmoid"):
        s.lr = lr
        s.size = my_size
        s.prev_size = prev_size
        s.weights = np.random.random((prev_size, s.size))
        funcs = activations_dict.get(activation_type)
        s.activation_f = funcs[0]
        s.d_activation_f = funcs[1]


    def activate(s, x):
        return s.activation_f(x)
        
    def d_activate(s, x):
        return s.d_activation_f(x)  

    def forward(s, inputs):
        s.input = np.asarray(inputs)
        s.neurons = np.dot(s.input, s.weights)
        s.neurons_activated = s.activate(s.neurons)
        
        return s.neurons_activated

    def forward_nu(s, inputs):
        input_data = np.asarray(inputs)
        neurons = np.dot(input_data, s.weights)
        neurons_activated = s.activate(neurons)
        
        return neurons_activated

    def backprop(s, layer_error): # for output here layer_error = target_val - s.forward
        s.delta = layer_error * s.d_activate(s.neurons_activated)
        s.prev_layer_error = np.dot(s.delta, s.weights.T)
        return s.prev_layer_error
        
    def update_weights(s):
        #print('input in layer ', s.input.T)
        s.input_t = s.input.T
        s.v = s.input_t.reshape((s.input_t.shape[0], 1))
        s.d = s.delta.reshape((1, s.delta.shape[0]))
        s.weights += s.lr * np.dot(s.v, s.d)
        
    def print_info(s):
        print("my size: ", s.size)
        print("w: ", s.weights, "\n")
        
    def print_pic(s):
        print_size = min(2, s.size)
        print("\n╻...\nv...")
        for i in range(print_size):
            #if (i == print_size/2 and print_size != s.size):
                #print ("--", s.size, "--\t", end='')
            print('O\t', end='')
        print ("--", format(s.size, ' 5d') , "--\t", end='')

    def add_neuron(s):     
        add_w = np.zeros(shape=(s.prev_size, 1), dtype=float) + 0.1
        s.weights = np.concatenate((s.weights, add_w.T))
        s.size+=1


class input_layer:
    def __init__(s, size):
        s.size = size
        s.values = np.zeros(shape=(size), dtype = float)

    def print_info(s):
        print("IN LAYER\nsize: ", s.size)
        print(s.values)

    def print_pic(s):
        print_size = min(2, s.size)

        for i in range(print_size):
            #if (i == print_size/2 and print_size != s.size):
                #print ("--", s.size, "--\t", end='')
                
            print("| |\t", end='')
        print("")
        for i in range(print_size):
            #if (i == print_size/2 and print_size != s.size):
                #print ("--", s.size, "--\t", end='')
            print(" v \t", end='')
        print("")
        for i in range(print_size):
            #if (i == print_size/2 and print_size != s.size):
                #print ("--", s.size, "--\t", end='')
            print(' @\t', end='')
        print ("--", format(s.size, ' 5d') , "--\t", end='')

    def forward(s, x):
        return x
        
    def forward_nu(s, x):
        return x
        
        


In [7]:

class plastic_nn:
    def __init__(s):
        s.layers = []
        s.n_of_layers = 0
        s.name = 'noname'
        pass

    def give_name(s, name):
        s.name = name
        
    def deep_copy(s):
        return copy.deepcopy(s)

    def forward(s, data):
        for lay in s.layers:
            data = lay.forward(data)
        s.last_result = data
        return s.last_result

    def forward_nu(s, data):
        for lay in s.layers:
            data = lay.forward_nu(data)
        last_result = data
        return last_result
        
    
    def backprop(s, correct):
        err = correct-s.last_result

        for lay in reversed(s.layers[1:]):
            err = lay.backprop(err)

    def backprop_error(s, err):
        #err = correct-s.last_result

        for lay in reversed(s.layers[1:]):
            err = lay.backprop(err)

    def update_w(s):
        i = 0
        for lay in reversed(s.layers[1:]):
            #print('layer idx: ', i)
            i+=1
            lay.update_weights()
            #print('\n')

    def learn_one(s, in_data, target_data):
        pnn.forward(in_data)
        pnn.backprop(target_data)
        pnn.update_w()   
        
    def append_one(s, new_layer, check = False):
        if check and s.n_of_layers!=0:
            last_layer_size = s.layers[-1].size
            if last_layer_size != new_layer.prev_size:
                print("size not match, layer ", s.n_of_layers)
                return
        s.layers.append(new_layer)
        s.n_of_layers+=1
        pass

    def check_layers_sizes(s, check_layers):
        for i in range(1, len(check_layers)):
            if (check_layers[i-1].size != check_layers[i].prev_size):
                print("error between ", i-1, "and ", i)
                return False
        return True
        
    def append(s, new_layers):
        test_layers = np.array([])
        if s.n_of_layers != 0:
            test_layers = s.layers[-1]
            
        test_layers = np.append(test_layers, new_layers)
                
        if (s.check_layers_sizes(test_layers)):
            for lay in new_layers:
                s.append_one(lay)
            print("added LAYERS succesfully")
            return True
        else:
            print("ERROR adding layers, check info above")
            return False

    def print_info(s):
        print('NAME: ', s.name, ' (', s.n_of_layers, ')')
        for cnt in range(3,4): # s.n_of_layers
            print("#", cnt)
            s.layers[cnt].print_info()
            print("")
    
    def print_pic(s):
        print('NAME: ', s.name, ' (', s.n_of_layers, ')')
        cnt = 0
        for lay in s.layers:
            lay.print_pic()
            print("#", cnt, end='')
            cnt+=1
        print("\nOUT |#|\nOUT  v")


